In [1]:
import gensim
import gensim.downloader as api



w2v = api.load('word2vec-google-news-300')

for i, word in enumerate(w2v.vocab):
    if i == 10:
        break
    print(word)

</s>
in
for
that
is
on
##
The
with
said


# Setup data
### Combine the fake data and real data.
### Sample it and randomize.

In [9]:
import pandas as pd
from pandas import concat
import numpy as np
from sklearn.model_selection import train_test_split


fake_data = pd.read_csv("../data/Fake.csv")
real_data = pd.read_csv("../data/True.csv")

# Mark data as real or fake.
fake_data["real"] = 0
real_data["real"] = 1

df = pd.DataFrame(np.random.randn(100, 2))
msk = np.random.randn(len(df)) < 0.8

train_fake, test_fake = train_test_split(fake_data, test_size=0.25)
train_real, test_real = train_test_split(real_data, test_size=0.25)

test_frames = [test_fake, test_real]
train_frames = [train_fake, train_real]

train = concat(train_frames)
test = concat(test_frames)

train = train.sample(frac=1)
test = test.sample(frac=1)

## Replace text with vectors using google word2vec

In [19]:
import torchtext.data.utils as text_utils

tokenizer = text_utils.get_tokenizer("basic_english")
train_titles = train["title"]
train_text = train["text"]

# for title in train["title"]:
#     token_title = tokenizer(title)
for sentence in train_titles.to_list():
    print(tokenizer(sentence))

# i = 0
# for title in train["title"]:
#     if i > 10:
#         break
#     print(title)
#     i += 1

# for title in train["title"]:
#     for word in title:
#         w2v.vectors(word)

['war', 'crimes', 'court', 'backs', 'burundi', 'investigation']
['islamic', 'state', 'claims', 'air', 'base', 'attack', 'in', 'egypt', "'", 's', 'north', 'sinai']
['montel', 'williams', 'says', 'oregon', 'patriots', 'are', '“undereducated', 'terrorist', 'buffoons”', 'and', 'national', 'guard', 'should', '“shoot', 'to', 'kill”', 'them']
['german', 'police', 'arrest', 'man', 'after', 'posting', 'photo', 'of', 'child', 'abuse', 'victim']
['house', 'democrats', 'broadcast', "'", 'sit-in', "'", 'on', 'social', 'media', 'after', 'cameras', 'shut', 'down']
['u', '.', 's', '.', 'congress', 'to', 'act', 'on', 'puerto', 'rico', 'relief', 'house', 'speaker']
['islamic', 'state', 'claims', 'blast', 'at', 'afghan', 'intelligence', 'agency', 'in', 'kabul']
['house', 'reauthorizes', 'iran', 'sanctions', 'bill', ',', 'sets', 'syria', 'sanctions']
['not', 'so', '“funny', 'guys”', 'carl', 'reiner', 'and', '“meathead”', 'son', '“feel', 'sorry', 'for', 'obama”', 'after', '“racist”', 'americans', 'elected'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

